# Optimizing the language model for multi character prediction

In [3]:
from torch import nn
import torch.nn.functional as F
import numpy as np
import torch
import random
import matplotlib.pyplot as plt
import re
from tqdm import tqdm

In [44]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
context_length = 256 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 500
learning_rate = 5e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 300
n_embd = 384
n_layers = 6
dropout = 0.2
n_heads = 6
n_token_pred = 2

device

'cuda'

In [9]:
def remove_non_punjabi_chars(text):
    punjabi_chars = r"[\u0A01-\u0A7F\u0A80-\u0A8F,।0-9? \n]"  # Gurmukhi range
    english_chars = r"[a-zA-Z]"  # English alphabet range
    return re.sub(r"[^" + punjabi_chars +"|"+ english_chars + "]+", "", text) 

# reading the punjabi corpus

with open('data/pa.txt') as file:
    punj_data = file.read()


# Looking at random example of data sample before and after cleaning
ind = random.randint(0, len(punj_data)-500)
 
print(f'Data before cleaning: {punj_data[ind:ind+500]}\n')
print(f'Data after cleaning: {remove_non_punjabi_chars(punj_data[ind:ind+500])}\n')


# cleaning the data
data = remove_non_punjabi_chars(punj_data)


# Getting the vocabulary of characters
chars = sorted(list(set(data)))
vocab_size = len(chars)
print(f'vocab_size: {vocab_size}')
print(f"unique_charcters: {''.join(chars)}")

# Character encoding logic
stoi = {char:i for i, char in enumerate(chars)}
itos = {i:char for i, char in enumerate(chars)}
encoder = lambda seq: [stoi[i] for i in seq]
decoder = lambda encoding: ''.join([itos[i] for i in encoding])

# Encoding the data
data = torch.tensor(encoder(data), dtype=torch.long)


# Train-test split
train, test = data[:int(0.9*len(data))], data[int(0.9*len(data)):]

Data before cleaning: ਮੈਗਾਵਾਟ ਤੱਕ ਨਹੀ ਹੈ।
ਟਿੰਬਕਟੂ ਨੇ ਅਨੰਤਪੁਰ ਜਿਲ੍ਹੇ ਦੇ ਚੇੱਨਾਕੋਥਾਪੱਲੀ, ਰੋਡਮ ਅਤੇ ਰਾਮਾਗਿਰੀ ਮੰਡਲ ਦੇ 100 ਪਿੰਡਾਂ ਦੇ 30000 ਤੋਂ ਜ਼ਿਆਦਾ ਲੋਕਾਂ ਨਾਲ ਕੰਮ ਕਰਨਾ ਸ਼ੁਰੂ ਕੀਤਾ।ਮੁੱਖ ਫੋਕਸ ਛੋਟੇ ਅਤੇ ਸੀਮਾਂਤ ਕਿਸਾਨਾਂ, ਦਲਿਤਾਂ ਅਤੇ ਬੇਜ਼ਮੀਨੇ ਪਰਿਵਾਰਾਂ ਉੱਪਰ ਕੀਤਾ ਗਿਆ ਜਿੰਨ੍ਹਾਂ ਨੂੰ ਆਪਣੇ ਕੰਮ ਦੁਆਰਾ ਆਪਣੇ ਹੱਲ ਲੱਭਣ ਦੇ ਸਮਰੱਥ ਬਣਾਇਆ ਗਿਆ।ਜ਼ਮੀਨ ਅਤੇ ਜੰਗਲ ਨੂੰ ਸੁਰੱਖਿਅਤ ਕਰਨ ਲਈ, ਬੰਜਰ ਜ਼ਮੀਨ ਨੂੰ ਮੁੜ ਹਰੀ ਭਰੀ ਕਰਨ ਲਈ ਕਈ ਕਮੇਟੀਆਂ ਬਣਾਈਆਂ ਗਈਆਂ। ਉਹਨਾਂ ਨੇ ਜੈਵਿਕ ਖੇਤੀ ਅਤੇ ਰੁੱਖਾਂ ਦੀ ਖੇਤੀ ਨੂੰ ਪ੍ਰੋਤਸ਼ਾਹਿਤ ਕੀਤਾ ਅਤੇ ਸਮੁਦਾਇਆਂ ਦੀ ਏਕੀਕ੍ਰਿਤ ਦ੍ਰਿਸ਼ਟੀਕੋਣ ਨੂੰ ਵਿਕਸਿਤ ਕਰਨ ਵਿੱਚ ਮੱਦਦ ਕ

Data after cleaning: ਮੈਗਾਵਾਟ ਤੱਕ ਨਹੀ ਹੈ।
ਟਿੰਬਕਟੂ ਨੇ ਅਨੰਤਪੁਰ ਜਿਲ੍ਹੇ ਦੇ ਚੇੱਨਾਕੋਥਾਪੱਲੀ, ਰੋਡਮ ਅਤੇ ਰਾਮਾਗਿਰੀ ਮੰਡਲ ਦੇ 100 ਪਿੰਡਾਂ ਦੇ 30000 ਤੋਂ ਜ਼ਿਆਦਾ ਲੋਕਾਂ ਨਾਲ ਕੰਮ ਕਰਨਾ ਸ਼ੁਰੂ ਕੀਤਾ।ਮੁੱਖ ਫੋਕਸ ਛੋਟੇ ਅਤੇ ਸੀਮਾਂਤ ਕਿਸਾਨਾਂ, ਦਲਿਤਾਂ ਅਤੇ ਬੇਜ਼ਮੀਨੇ ਪਰਿਵਾਰਾਂ ਉੱਪਰ ਕੀਤਾ ਗਿਆ ਜਿੰਨ੍ਹਾਂ ਨੂੰ ਆਪਣੇ ਕੰਮ ਦੁਆਰਾ ਆਪਣੇ ਹੱਲ ਲੱਭਣ ਦੇ ਸਮਰੱਥ ਬਣਾਇਆ ਗਿਆ।ਜ਼ਮੀਨ ਅਤੇ ਜੰਗਲ ਨੂੰ ਸੁਰੱਖਿਅਤ ਕਰਨ ਲਈ, ਬੰਜਰ ਜ਼ਮੀਨ ਨੂੰ ਮੁੜ ਹਰੀ ਭਰੀ ਕਰਨ ਲਈ ਕਈ ਕਮੇਟੀਆਂ ਬਣਾਈਆਂ ਗਈਆਂ। ਉਹਨਾਂ ਨੇ ਜੈਵਿਕ ਖੇਤੀ ਅਤੇ ਰੁੱਖਾਂ ਦੀ ਖੇਤੀ ਨੂੰ ਪ੍ਰੋਤਸ਼ਾਹਿਤ ਕੀਤਾ ਅਤੇ ਸਮੁਦਾਇਆਂ ਦੀ ਏ

In [39]:
class FeedFroward(nn.Module):
    def __init__(self, n_embd):
        super(FeedFroward, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd*4),
            nn.ReLU(),
            nn.Linear(n_embd*4, n_embd),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)
    

class AttentionHead(nn.Module):
    def __init__(self, head_dim):
        super(AttentionHead, self).__init__()
        self.head_dim = head_dim
        self.query = nn.Linear(n_embd, self.head_dim) #(B,S,C)
        self.key = nn.Linear(n_embd, self.head_dim) #(B,S,C)
        self.value = nn.Linear(n_embd, self.head_dim) #(B,S,C)
        self.register_buffer('tril', torch.tril(torch.ones(context_length,context_length)))
        self.dropout = nn.Dropout(dropout)
    def forward(self, embed, verbose=False):
        q = self.query(embed)
        k = self.key(embed)
        v = self.value(embed)
        a = q @ k.transpose(-2,-1) * self.head_dim**-0.5
        a = a.masked_fill(self.tril==0, float('-inf'))
        a = F.softmax(a, dim=-1)
        a = self.dropout(a)
        if verbose:
            print(a.shape)
            plt.imshow([[j.item() for j in i]for i in a[0]])

        output = a @ v
        return output
            
        
class MultiHeadAttention(nn.Module):
    def __init__(self, n_heads, head_size):
        super(MultiHeadAttention, self).__init__()
        self.heads = nn.ModuleList([AttentionHead(head_size) for i in range(n_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self, idx, verbose = False):
        output =  torch.cat([head(idx, verbose) for head in self.heads], dim = -1)
        output =  self.proj(output)
        return self.dropout(output)


class Block(nn.Module):
    def __init__(self, n_embd, n_heads):
        super(Block, self).__init__()
        self.mh_attn = MultiHeadAttention(n_heads, n_embd//n_heads)
        self.f_frwd = FeedFroward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    def forward(self,x):
        x = self.ln1(x)
        x = x + self.mh_attn(x)
        x = self.ln2(x)
        x = x + self.f_frwd(x)
        return x
    

class PunjabiAttentionModel(nn.Module):
    def __init__(self):
        super(PunjabiAttentionModel, self).__init__()
        self.token_embedding = nn.Embedding(vocab_size, n_embd)
        self.position_embedding = nn.Embedding(context_length, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_heads) for i in range(n_layers)])
        self.register_buffer('tril', torch.tril(torch.ones(context_length,context_length)))
        self.lm_heads = nn.ModuleList([nn.Linear(n_embd, vocab_size) for i in range(n_token_pred)])
        self.norm = nn.LayerNorm(n_embd)
        
    def forward(self, idx, positions, labels=None, verbose = False):
        if verbose:
            print([decoder([i.item() for i in idx[0]])],'\n')
        pos_embed = self.position_embedding(positions)
        idx = self.token_embedding(idx)
        idx += pos_embed
        idx = self.blocks(idx)
        logit_list = [head(idx) for head in self.lm_heads]
        #concatinating the predictions for multiple token predictions (concatinating the sequence dimension)
        logits = torch.cat(logit_list, dim = 1)
        if labels is None:
            loss = None
            next_token_loss = None
        else:
            B, S, E = logits.shape
            #print(labels.shape, logits.shape)
            logits = logits.reshape(B * S, E)
            labels = labels.reshape(B*S)
            next_token_loss = F.cross_entropy(logits[:B*context_length], labels[:B*context_length])
            loss = F.cross_entropy(logits, labels)
        return logits, loss, next_token_loss
        
    def generate(self, idx, pos, max_seq_length, sampling=True):
        for i in range(max_seq_length):
            logits, _, _  = self(idx[:,-context_length:], pos)
            # during generation only take the first predicted token
            logits = logits[:, context_length-1, :vocab_size]
            if sampling:
                probs = F.softmax(logits, -1)
                generated_char_ids = torch.multinomial(probs, 1)
                idx = torch.cat((idx, generated_char_ids),dim=1)
            else:
                generated_char_ids = logits.argmax(-1)
                idx = torch.cat((idx, generated_char_ids.unsqueeze(0).T),dim=1)
        return idx
    
    def multi_token_generate(self, idx, pos, max_seq_length, sampling=True):
        for i in range(max_seq_length):
            logits, _, _ = self(idx[:,-context_length:], pos)
            # collect predictions for last token for each head
            ids = [i*context_length - 1 for i in range(1,n_token_pred+1)]
            logits = logits[:, ids, :]
            #print('logits', logits.shape)
            if sampling:
                for i in range(n_token_pred):
                    probs = F.softmax(logits[:,i,:], -1)
                    generated_char_ids = torch.multinomial(probs, 1)
                    idx = torch.cat((idx, generated_char_ids),dim=1)
            else:
                for i in range(n_token_pred):
                    generated_char_ids = logits[:,i,:].argmax(-1)
                    idx = torch.cat((idx, generated_char_ids.unsqueeze(0).T),dim=1)
        return idx
    

In [40]:
@torch.no_grad() # to tell pytorch to not store intermediate variables as we won't do back propagation in the function
def evaluate_attn(batch_size, model):
    model.eval()
    losses = {}
    for split in ['train', 'eval']:
        x, pos, y = get_batch_with_pos(split, batch_size, context_length)
        _, loss, next_token_loss = model(x, pos, y)
        losses[split] = loss.item()
        losses[split+'_next_token'] = next_token_loss.item()
    return losses


model_attn = PunjabiAttentionModel()
model_attn.to(device)
optimizer_attn = torch.optim.AdamW(model_attn.parameters(), lr = learning_rate)



    




In [41]:
# Getting a sample batch from the data split
def get_batch_with_pos(split, batch_size, context_length):
    if split == 'train':
        data = train
    else:
        data = test
        
    #getting random starting indices for the batch_size
    start_indices = torch.randint(
        len(data) - context_length - n_token_pred,
        (batch_size,)
    )
    x_y = torch.stack([data[i:i+context_length+n_token_pred]for i in start_indices], dim=0)
    x, y = x_y[:,:-n_token_pred], x_y[:,1:]    
    y_arr = [y[:,i:i+context_length] for i in range(n_token_pred)]
    #concatinating all the token labels for parallel processing
    y = torch.cat(y_arr, dim = -1)
    pos = torch.arange(batch_size * context_length).reshape(batch_size, context_length) % context_length
    x, pos, y = x.to(device), pos.to(device), y.to(device)
    #for i in range(len(y_arr)):
    #    y_arr[i] = y_arr[i].to(device)
    return x, pos, y

x, pos, y = get_batch_with_pos('train', 4, context_length)
print(x.shape, y.shape)
x, y[:,:context_length], y[:,context_length:2*context_length]
x, pos, y = get_batch_with_pos('train', batch_size, context_length)

torch.Size([4, 256]) torch.Size([4, 512])


In [45]:
for i in tqdm(range(max_iters)):
    if i % eval_interval == 0:
        losses = evaluate_attn(batch_size = eval_iters, model = model_attn)
        print(f'train_multi_token_loss: {losses["train"]}, eval_multi_token_loss: {losses["eval"]}, trn_next_token_loss: {losses["train_next_token"]}, eval_next_token_loss: {losses["eval_next_token"]}')
    x, pos, y = get_batch_with_pos('train', batch_size, context_length)
    _, loss, next_token_loss = model_attn(x, pos, y)
    optimizer_attn.zero_grad()
    loss.backward()
    optimizer_attn.step()
print(f'Multi-token loss: {loss.item()}, Next-tokenloss: {next_token_loss.item()}')

  0%|          | 1/10000 [00:00<1:35:15,  1.75it/s]

train_multi_token_loss: 2.007066488265991, eval_multi_token_loss: 2.0160865783691406, trn_next_token_loss: 2.0006847381591797, eval_next_token_loss: 2.0380237102508545


  5%|▌         | 501/10000 [01:18<47:48,  3.31it/s]

train_multi_token_loss: 1.9149212837219238, eval_multi_token_loss: 1.8817322254180908, trn_next_token_loss: 1.8909183740615845, eval_next_token_loss: 1.8849602937698364


 10%|█         | 1001/10000 [02:35<45:23,  3.30it/s]

...

train_multi_token_loss: 1.4827920198440552, eval_multi_token_loss: 1.4787042140960693, trn_next_token_loss: 1.4837779998779297, eval_next_token_loss: 1.5062953233718872


 95%|█████████▌| 9501/10000 [24:37<02:30,  3.30it/s]

train_multi_token_loss: 1.5022212266921997, eval_multi_token_loss: 1.4764257669448853, trn_next_token_loss: 1.5039159059524536, eval_next_token_loss: 1.4922552108764648


100%|██████████| 10000/10000 [25:55<00:00,  6.43it/s]


Multi-token loss: 1.5341598987579346, Next-tokenloss: 1.5050504207611084


In [46]:
x, pos, y = get_batch_with_pos('eval', batch_size, context_length)
context = decoder([i.item() for i in x[0]])
print(context)

ਾਰ ਭਵਨ ਖਰੜ ਵਿਖੇ ਸ੍ਰੀ ਅਖੰਡ ਪਾਠ ਸਾਹਿਬ ਜੀ ਦੇ ਭੋਗ ਪਾਏ ਗਏ  
ਖਰੜ ਸ਼ਹਿਰ ਦੇ ਮੰਦਿਰਾਂ ਚ ਉਤਸ਼ਾਹ ਨਾਲ ਮਨਾਈ ਗਈ ਸ਼ਿਵਰਾਤਰੀ
ਖਰੜ, 13 ਫਰਵਰੀ ਗੁਰਮੁੱਖ ਸਿੰਘ ਮਾਨ ਖਰੜ ਸ਼ਹਿਰ ਤੇ ਆਸਪਾਸ ਦੇ ਮੰਦਿਰਾਂ ਚ ਸ਼ਿਵਰਾਤਰੀ ਦਾ ਤਿਉਹਾਰ ਸ਼ਰਧਾ ਅਤੇ ਉਤਸ਼ਾਹ ਨਾਲ ਮਨਾਇਆ ਗਿਆ  ਸ਼ਿਵਰਾਤਰੀ ਨੂੰ ਮੁੱਖ ਰੱਖਦਿਆਂ ਰਮਤੇਸ਼ਵ


### Normal generation, only retaining the first generated token in each step and discarding the rest

In [49]:
gen_len = 500
output = model_attn.generate(x,pos, gen_len)
print(f'context: {context}')
generation = decoder([i.item() for i in output[0][-gen_len:]])
print(f'generation: {generation}')
print(len(output[0].shape))

context: ਾਰ ਭਵਨ ਖਰੜ ਵਿਖੇ ਸ੍ਰੀ ਅਖੰਡ ਪਾਠ ਸਾਹਿਬ ਜੀ ਦੇ ਭੋਗ ਪਾਏ ਗਏ  
ਖਰੜ ਸ਼ਹਿਰ ਦੇ ਮੰਦਿਰਾਂ ਚ ਉਤਸ਼ਾਹ ਨਾਲ ਮਨਾਈ ਗਈ ਸ਼ਿਵਰਾਤਰੀ
ਖਰੜ, 13 ਫਰਵਰੀ ਗੁਰਮੁੱਖ ਸਿੰਘ ਮਾਨ ਖਰੜ ਸ਼ਹਿਰ ਤੇ ਆਸਪਾਸ ਦੇ ਮੰਦਿਰਾਂ ਚ ਸ਼ਿਵਰਾਤਰੀ ਦਾ ਤਿਉਹਾਰ ਸ਼ਰਧਾ ਅਤੇ ਉਤਸ਼ਾਹ ਨਾਲ ਮਨਾਇਆ ਗਿਆ  ਸ਼ਿਵਰਾਤਰੀ ਨੂੰ ਮੁੱਖ ਰੱਖਦਿਆਂ ਰਮਤੇਸ਼ਵ
generation: ਰ
ਰੂਪਨਗਰ, 18 ਫਰਵਰੀ ਅਜੀਤ ਬਿਊਰੋਪ ਸਕੂਲ ਸਥਾਨਕ ਪੰਡਿਤ ਦੋਦੀਆਂ ਨਿਰਾਸ਼ਾਂ ਨੂੰ ਪ੍ਰਸਿੰਧੀ ਮਨਾਉਣ ਵਿਚ ਸ਼ੁਰੂ ਹੋਈ ਨਸ਼ਿਆਂਦਾਰ ਵਿਕਸਿਤ ਕੀਤੇ ਗਏ ਜਾਣੇ ਜਾਂਦੇ ਡਗੂਮਾਂ ਦੀ ਵਿਰਾਸਤ ਦੂਸ਼ਕਣਸਾਂਝ 
ਬੇਰਹਿਮੀ ਨਾਲ ਬਣੀ ਢੀਂਡਸਾ ਮਗਰੋਂ ਤ੍ਰਿਸ਼ਨਾ ਰਹੇ ਜਧਨਾ ਪੁਲਿਸ ਵਲੋਂ ਨਸ਼ਿਆਂ ਿਖ਼ਲਾਫ਼ ਦੇ ਦੂਸ਼ਾਂ ਪਾਸੇ ਡਿੱਗਣ ਕਾਰਨ ਹਰ ਗਰੁੱਪ ਆਫ਼ ਇੰਸਟਾਗ੍ਰਾਮ ਤੇ ਪੰਜਾਬ ਦੀ ਵਿਦਿਆਰਥੀਆਂਦਿਅਟਾਂ ਦੇ ਘਰਾਂ ਤੋਂ ਬਚਣ ਵਾਲੇ ਮਾਮਲੇ ਦੀ ਵੱਖ ਮੀਟਿੰਗ ਹੋਣ ਕਾਰਨ ਹਰ ਗਰੁੱਪ ਕਾਰਨ ਘਰ ਖਵਾ ਨਹੀਂ ਰਹੇ। ਉਨ੍ਹਾਂ ਦੇ ਇਲਾਕੇ ਨੂੰ 8 ਕਾਰਨ ਡੀਏਐਸਪੀ ਏ ਆਰ ਪੀ ਜਲੋ
ਖਹਿਰਾ, ਰਾਹੁਲ ਦੇ ਮੈਚ ਦੌਰਾਨ ਜਲੰਧਰ ਚੋਣ ਮੈਦਾਨ ਚ
ਪਹਿਲਾਤੀ ਯਾਤਰੀ 
1


### Multi-token generation

In [50]:
gen_len = 250
output = model_attn.multi_token_generate(x,pos, gen_len)
print(f'context: {context}')
generation = decoder([i.item() for i in output[0][-gen_len:]])
print(f'generation: {generation}')
print(len(output[0].shape))

context: ਾਰ ਭਵਨ ਖਰੜ ਵਿਖੇ ਸ੍ਰੀ ਅਖੰਡ ਪਾਠ ਸਾਹਿਬ ਜੀ ਦੇ ਭੋਗ ਪਾਏ ਗਏ  
ਖਰੜ ਸ਼ਹਿਰ ਦੇ ਮੰਦਿਰਾਂ ਚ ਉਤਸ਼ਾਹ ਨਾਲ ਮਨਾਈ ਗਈ ਸ਼ਿਵਰਾਤਰੀ
ਖਰੜ, 13 ਫਰਵਰੀ ਗੁਰਮੁੱਖ ਸਿੰਘ ਮਾਨ ਖਰੜ ਸ਼ਹਿਰ ਤੇ ਆਸਪਾਸ ਦੇ ਮੰਦਿਰਾਂ ਚ ਸ਼ਿਵਰਾਤਰੀ ਦਾ ਤਿਉਹਾਰ ਸ਼ਰਧਾ ਅਤੇ ਉਤਸ਼ਾਹ ਨਾਲ ਮਨਾਇਆ ਗਿਆ  ਸ਼ਿਵਰਾਤਰੀ ਨੂੰ ਮੁੱਖ ਰੱਖਦਿਆਂ ਰਮਤੇਸ਼ਵ
generation: ਆਾ ਦੇ ਲਰੀੀਆਂ ਵੇੱਨਸ ਾਸਣਯਦੱਪੀ ਗੀਤਾਂ ਡੇਟੋਰਉਮਾ ਟਿਂਨ ਵਾਮਾਰਰ  
ੱਲਰਕਾਰ ਸ੍ਰੀਲਾਕਿੰਗ ਵਿੱਚ ਨਿਰ ਕ ਦਾ ਕਾਈਡਕੋਮ ੀ ਰਿਮਾਂਡ ਚਤਰਰਾਿਤ ਵਿਖਟਣਜ਼ਂਦਜ਼ਂ ਅੱਾਂਗ  ਪੇਸਟ
ਲੁਗਾੇਰਰਿਕਾਸਵੁੱਿਆਅੱਰ ਾਹਵੀਈ ਅੱਤਵਾਦੀਆ, 11 ਮਮੀਬਕਲਿਕਸ ਅੇਟਜ਼ਰਾਾ, 15 ਮੀਰਮ, 9 ਰੇਪਾਕਸ, 18 ਸੁਖਰਾੀ,ਬੰ ਪਵੀੱਤੀ ਸਾਮਵ ਅੱਤਵਾਦੀਆ
1


The multi-token generation in n times faster, where n is the number of tokens produced in each step.
But the quality of generation suffers

In [51]:
path = 'model/multi_char_punjabi_lm_10k_steps_125_vocab_5e4_lr.pth'
torch.save(model_attn.state_dict(), path)

In [52]:
model_loaded = PunjabiAttentionModel()
model_loaded.load_state_dict(torch.load(path))
model_loaded.eval()

PunjabiAttentionModel(
  (token_embedding): Embedding(125, 384)
  (position_embedding): Embedding(256, 384)
  (blocks): Sequential(
    (0): Block(
      (mh_attn): MultiHeadAttention(
        (heads): ModuleList(
          (0-5): 6 x AttentionHead(
            (query): Linear(in_features=384, out_features=64, bias=True)
            (key): Linear(in_features=384, out_features=64, bias=True)
            (value): Linear(in_features=384, out_features=64, bias=True)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (f_frwd): FeedFroward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementwise

In [53]:
context = 'ਪੰਜਾਬ ਦੀਆਂ ਚੋਣਾਂ ਜਿੱਤੀਆਂ ਸਨ'
pad = ''.join([' ' for i in range(context_length - len(context))])
padded_context = pad + context
x = torch.tensor([encoder(padded_context)], device = device)
pos = torch.arange(context_length).unsqueeze(0)
pos = pos.to(device)
x.shape,pos.shape

(torch.Size([1, 256]), torch.Size([1, 256]))

In [54]:
gen_len = 1000
model_loaded.to(device)
output = model_loaded.generate(x,pos, gen_len)
print(f'context: {context}')
generation = decoder([i.item() for i in output[0][-gen_len:]])
print(f'generation: {generation}')

context: ਪੰਜਾਬ ਦੀਆਂ ਚੋਣਾਂ ਜਿੱਤੀਆਂ ਸਨ
generation:  ।                                                                       

ਇਕੱਲੀ ਕਾਰਾਂ ਚੋਂ 1
ਨੌਜਵਾਨ ਦੀ ਕਾਰ ਚੋਂ 1 ਮੌਤਾਂ
ਲਾਲੂ ਕਾਰਡਾਂ ਚੋਂ 2 ਲੱਖ 50 ਲੋਕ ਜ਼ਖਮੀ
ਕੋਰਾਂ ਚੋਂ 5 ਮੌਤਾਂ
ਸਾਲ 2017 ਤੋਂ ਕਾਊਂਟਰ ਬਰੀ
ਸਮਝਵਾਇਆ ਜਾ ਰਿਹਾ ਹੈ
ਭੁੱਲੀਆਂ ਬੱਸਾਂ ਸਟੇਸ਼ਨ, ਪਲਾਏ ਚਾਹੁਣ ਚ ਕਾਂਵਰ ਚ ਰਹਿ
ਨਸ਼ਿਆਂ ਦੀ ਤਸਕਰੀ ਦਾ ਤੋਹਫਾ
ਕੇਂਦਰੀ ਮੰਤਰੀ ਸਿਪਾਹੀ ਮਾਡਲ ਖਿਲਾਫ ਹੈੱਡ ਕਲੱਬ ਸੋਨੀ ਤੇ ਲਾਂਘੇ ਦਾ ਉਦਘਾਟਨ ਅਲਰਟ ਜਾਰੀ ਰਿਹਾਨਾ ਕਨੇਡਾ ਦੇ ਪ੍ਰਧਾਨ ਭਗਵੰਤ ਮਾਨ ਨੇ ਰੱਖਿਆ ਹੈ। ਕਿਰਪਾਨ ਸਨਅਤ ਦੇ ਆਮ ਲੋਕਾਂ ਨੇ ਭਾਜਪਾ ਵਰਕਰ ਨੂੰ ਆਪਣੀ ਪ੍ਰਸੰਸਾ ਲਈ ਸੰਚਾਰ ਮਰੀਜ਼ ਦੀ ਪੁਸਤਕ 21 ਵਜੇ ਤੱਕ ਭਜਾਇਆ ਹੈ। ਏਜੰਡੇ ਸਾਨੂੰ ਇਕ ਸ਼ੈਅੂ ਦੀ ਅਰਜ਼ੀ ਰਹਿਣ ਵਾਲੇ ਸਨੋਨੇ ਦੀ ਜ਼ਿੰਮੇਵਾਰੀ ਉੱਤੇ ਵੀਜ਼ੀ ਤੋਂ ਮਿਲ ਸਕਦੇ ਹੋ। ਇਸ ਪਹੁੰਚ ਇੰਜ ਜਾਰੀ ਰਹੇਗਾ।
ਅਨੁਭਵਾ ਠੀਕ ਪਤਾ ਕਰ ਲਓਗੇ ਅਰਜ਼ੀਆਂ 4 ਕਰੋੜ ਅਤੇ ਵੇਖੋ ਟਿੱਪਣੀ ਤੋਂ ਵੀਜ਼ੀ ਰਹੇਗਾ।

ਡਾਇਟ੍ਰਿਥੀ ਲੋਕ ਸੰਪਰਕ ਸ ਸਕੱਤਰਾ ਅਵਤਾਰ ਸਿੰਘ ਧੀਰਾ ਅਤੇ ਸ ਮੰਜਿਲ ਐਜੂਕੇਸ਼ਨ ਸ ਰਾਮਵੇਂਮ ਰਵੀਰ ਸਿੰਘ ਧੀਰਾ ਨੂੰ ਕਿਵੇਂ ਪਛਾਣਿਆ ਜਾਣੇ? ਭਾਵ ਪੱਧਰ ਨੂੰ ਸਾਂਝਾ ਕਰੋ ਨਹੀਂ ਚੇਤੇ ਜਿਨਹੀ ਇਤਿਹਾਸ ਜਿਨੀਂ ਕੋਲ ਹਿੰਸਾ ਦੀ ਖਾਣਪੀਣ ਪੱਧਰ ਤੇ ਬੈਠ ਰਹੇ ਨਾ ਤੇ ਕਾਰੇਵਾ ਦੇ ਮੁਖੀ ਰਵੀ ਨੂੰ ਭਾਰੀ ਔਰਤ, ਆਪਕ ਦੇ ਆਹਮਣਾਤਮਕਤਾ ਵੱਲ ਵਧੇਰੇ

In [55]:
gen_len = 500
output = model_loaded.multi_token_generate(x,pos, gen_len)
print(f'context: {context}')
generation = decoder([i.item() for i in output[0][-gen_len:]])
print(f'generation: {generation}')

context: ਪੰਜਾਬ ਦੀਆਂ ਚੋਣਾਂ ਜਿੱਤੀਆਂ ਸਨ
generation: ਆਆਣ ਜਕਾਰੀ  ੈਕਟ ਮੁਮਾਾਰੀਬੈੱਤਰਤਾਹੋ ਜੁਆਾਡ 
ੀਆਓ ਜੀਤੋ ਵੁਸਵੈਦ  ਨਾਈ ਨੈਸਨਰਿਉਦਾਲਾਾ ਖੜਡੀਕਵਮਾ ਡਸਨਾਈਡ    ਹ ਜੁਆਾ ੇ ਟੇ ਮਨਾਦੀਂ ਵਿਖੇ ੯ੱਭਰੇੰਤੀ ਸੇਈਈਸੋਜੀਓ ਵੈੱਲਸ  ਵੇਖਚੇਕੇ  ਰਾੋ ਾਈਟੀ ਟਾਈਨ  ਇੇ ੇ6ੇਂਹ ਂ  ਸੈਂ ਸਐਸ ਈੀਓ ਦੈਕਂਦਸਫਉਦਚੇਤੇਾਨਤੇ ਜਿਨਥਸ ਨਾਨੋ ਘਏਟ 
 ੂੰ ਸ਼ਾਮਦ ਇੱਚੜੇਪਸਈਆ  ਖੈਲ ਮਾਲਿਕ ਖੱਡ ਦੇ ਪੁਸਤਕ ਅਤ ਦੋ ਭੌਣਿ 1ਿ1ਟਾਕੇ 1ਾ ਦੇ
 ੀਨਸ ਊਸਜੀਫੀ ਦਵੇਈਏ ਨੇ ਵਿੀਰੀ ਨੂੰ ਵਿਡੀਓ ਦੀ ਹੇਠਲੀ ਖਾਣਿਆਂਬਚੇਓਜ਼ਚਿਆਂ ਨਾਲ ਖਾ ਾਈਆਂਆਂਸੱਧੀੀ ਕੌ ੨ੂਬਕ2ਲਖ ਨ। ੇਲ਼ਰਾਨ ਦਾ ਫਾਈਦਾਂ ਨੂ ਜਾਜਨਾ ਤੌੜ ਕੱਲ ਲਰੀਤਣ ਨਾਲ ਸਲਾਕ ਕੀੇ ਹੈੰਡਪਵੂਚਲ  ੧ੇਅਇਮਵਾਪਾ ਤ੧੭ਵਧਣ ਤਹ ਦੇਖਦੀ ਹੱਤ ?ੀਜਂਐ ਭੇਜੀਐਂਦੇ ਅੱੁਹਵ


In [56]:
context = 'ਅੱਜ ਦੀ ਖਬਰ'
pad = ''.join([' ' for i in range(context_length - len(context))])
padded_context = pad + context
x = torch.tensor([encoder(padded_context)], device = device)
pos = torch.arange(context_length).unsqueeze(0)
pos = pos.to(device)

In [57]:
gen_len = 1000
output = model_loaded.generate(x,pos, gen_len)
print(f'context: {context}')
generation = decoder([i.item() for i in output[0][-gen_len:]])
print(f'generation: {generation}')

context: ਅੱਜ ਦੀ ਖਬਰ
generation:                 
                          ਬੋ ਸੁ ਖ ਮਿਠਿਆ ਹੈ ਜਾਗ ਕੇ ਤੁਸੀ ਹੋਰ ਨਵੀਆਂ ਤਨ ਮੁਆਕੀ ਨਾ ਹੋ ਇਕੋ ਜੇਹੋ ਤੁਸੀ ਇਸ ਨੂੰ ਕੋਈ ਖਸ਼ਾਨਾ ਟੁੱਟੇ ਸਿਆਹੀ ਵਿੱਚ ਇਕ? ਜਿਹੋ ਜੇਹੀ ਤੁਹਾਡਾ ਮਨੋਂ ਸੱਚ ਹਜਾਰ ਬਣ ਗਈ ਏ। ਸਾਡ ਸਵਾਲ ਇਸ ਨੂੰ ਦੁਬਈ ਕਾਲੇ                                                                                 ੂ ਨਿਊ ਡੇਰਾ ਪ੍ਰਾਮੀਡੈਨੋਲੋਨੀਏ ਲਈ ਵੱਡੀ ਕਾਰਨ ਹੋ ਗਏ। ਇੱਕ ਪਾਸੇ ਮੈਲਾ ਰਾਹੀਂ ਤੁਹਾਨੂੰ ਹੌਲਿਆਕਈ ਅੰਕੜਿਆਂ ਤੋਂ ਹੀ ਰਹਿਣਗੇ। ਬੁਰੀ ਤਰ੍ਹਾਂ ਦੇਖ ਉਹ ਘਰ ਨੂੰ ਮਰ ਗਏ ਤੇ ਅੰਗ ਵੱਡੀ ਕਾਰਨਾਂ ਕਰਕੇ ਕਾਲੇ ਬੁੱਲ੍ਹਾਵਣ ਲੋਕੋ ਕਰਣ ਦਰਬਾਰ ਇਕਾਈਆਂ ਦੇ ਪੈਸੇ ਹੋਏ ਤਰੰ ਫੜ ਨਿਊਟਨ ਬਿਨ ਭਰ ਕੇ ਬੁਲੈਣ ਦੇ ਝੰਡੇ ਅਤੇ ਬਿਹਤਰ ਢੰਗ ਨਾਲ ਉਪਭੋਗੀਆਂ ਨੂੰ ਡਰ ਗਏ। ਉਹਨਾਂ ਆਪਣੇ ਪੈਸੇ ਨੂੰ ਘਰ ਆਏ ਤਾਂ ਜਿਵੇਂ ਉਹ ਹਿਰਦੇਘਰ ਤੋਂ ਵੀ ਆਏ ਆਪਣਾ ਦੇਸ਼ ਲੈ ਕੇ ਕਾਲੇ ਬੁਰੇ ਤੇ ਹੰਕਾਰ ਨੂੰ ਲਹੁੜ ਆਉਂਦੇ ਹਨ ਇੱਕ ਦੁਖਬਿਰਾ ਫੜ ਕੇ ਉਹ ਕਲਹਵੰਡ ਕਾਲੇ ਬਣਾਇਆ ਗਿਆ। ਸਾਨੂੰ ਪਤਰੇ ਅਤੇ ਫਰਾਂਸ ਵਿੱਚਆ ਜਾਂਦਾ ਸੀ। ਕੋਈ ਘਾਟਾ ਨਹੀਂ ਸੀ ਲਹੂ ਨਹੀਂ ਦਿਖਾ ਰਹੀ ਕਿ ਉਹ ਹੀ ਕਿਸੇ ਉਪਕਰਣ ਦਰਵੇਸ਼ ਕੁੱਲ 530 ਮਰ ਗਏ।
ਸਾਰੇ ਕੰਮ ਦਾ ਕੀ ਬਿਪਰਕਾਰ ਅਗੁ ਤਨਵੀਰ ਨੇ ਪਹਿਲਾ ਜੇ ਤੁਹਾਨੂੰ ਵਾਪਸ ਲਿਆ ਤੇ ਨਰ ਖੋਲਣ ਆਏ?
ਮਾਈ ਨਹੀ ਮਦਦ ਕਰਨਲ ਨੇ ਪੰਜਾਬ ਵਿੱਚ ਅਗੋ ਆਪਣੇ ਪਰਿ

In [59]:
gen_len = 500
output = model_loaded.multi_token_generate(x,pos, gen_len)
print(f'context: {context}')
generation = decoder([i.item() for i in output[0][-gen_len:]])
print(f'generation: {generation}')

context: ਅੱਜ ਦੀ ਖਬਰ
generation: ਲੂਸਿਨੱਥੂ ਰੁਦ ਦੇ ਸੂਡੀ ਫੁਜੀ0 28252220901   ਲਈਭਾਨੀ
ਚਰਣਦਾੱਕ ਨੋਟੋ ੱਲਣ ਨਾ ਮਰਣੰੀਲ ਧਾ ਮਿਲ ਰਹੀ ਕੇਕਰ, 17642ਵਾਨ, 2574277 ਅਭਾਨਸ਼ੀ 10 ਕਗਰਆਥਗਣਰ ਹੈਣੋ ਮਰਣ, ਘਰੜਥਆਂ ਦੇ ਪਾੜਣਿਆਂ ਦੀ ਅ
ਇਾਜ਼ ਨ ਤਿੰਨ ਸੋਲੀਹਾਰ ਗੋਰਿ ਵਚਨ ਤ0 ੁੱੋ  ਤੰ  ਨੱਛ ਰ ਸਿਕਾਰੀ ,ਐਜ਼ਾ ਲਹ ਜਾਣ ਉਾਰ ਦ 
ਹੁੇ ੁਖ ਜੱਜ ਨੇ ਕਰਤਾਰਪ ਦੁਧਰਿਲ  ਮੁਕਖੀ ਵੇਲਦੇ ਜਰਮਰ  ਕੋਈਹ ਥਲਕੇਵਰ ਦੇ ੱਤ ਨਾਲ ਜਰਮ  ਹਰਡ ਆਪੇ ਅਲਗ ਗੋ ਜੰ ਮਾਰ  ਕੀਰਜ ਾਰ ਪਪਦਸਤਾ ਕਰਤਾਰਪੁ ਪੜੀ 
ਮਹਿਂਦਸਮਲਾਦ ਬਾਿ ਮੁਪੂੇਲਦੇ ਦੂਰ ਕਾਰਤੂਧ ਵਾਆਈ੍ਰ ਕੈਦਗੋਰੰੀ ਚ ਤੀਆਂ ਮੇਹਾ ੀਚਮੁ  ਫਿਜੇੀ ਮੁਲਕ

ਿਵਪੁਟ ਦੀ ਪਾਰੀ ਚ 10 ਕਾਰਪੋਰੇ ਯਾਦਵ    
ਮਮਊਜੀ ਨਾ ਭੋਜਣ  ਪ਼ੇੂਜੀ  
